In [37]:
import requests_html
from requests_html import HTMLSession
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import re

In [5]:
cookies = {
    '__ddg1_': 'SXqUPtcG9xgXD7ofoyJN',
    '_gid': 'GA1.2.159272066.1699977138',
    'PHPSESSID': '9l6r04c7ajcoh4thoogpte39dp',
    'device_view': 'full',
    'WhiteCallback_visit': '23748250360',
    '_ym_uid': '1699977140598253825',
    '_ym_d': '1699977140',
    '_ym_isad': '2',
    '_ym_visorc': 'b',
    'g_state': '{"i_p":1699984347826,"i_l":1}',
    'WhiteCallback_visitorId': '14420866414',
    'WhiteSaas_uniqueLead': 'no',
    '_ga': 'GA1.1.2065722076.1699977138',
    '_ga_G3QSYZZMND': 'GS1.1.1699977138.1.1.1699981404.0.0.0',
}

headers = {
    'authority': 'www.restoclub.ru',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': '__ddg1_=SXqUPtcG9xgXD7ofoyJN; _gid=GA1.2.159272066.1699977138; PHPSESSID=9l6r04c7ajcoh4thoogpte39dp; device_view=full; WhiteCallback_visit=23748250360; _ym_uid=1699977140598253825; _ym_d=1699977140; _ym_isad=2; _ym_visorc=b; g_state={"i_p":1699984347826,"i_l":1}; WhiteCallback_visitorId=14420866414; WhiteSaas_uniqueLead=no; _ga=GA1.1.2065722076.1699977138; _ga_G3QSYZZMND=GS1.1.1699977138.1.1.1699981404.0.0.0',
    'pragma': 'no-cache',
    'referer': 'https://www.restoclub.ru/spb/search',
    'sec-ch-ua': '"Chromium";v="118", "Opera";v="104", "Not=A?Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 OPR/104.0.0.0 (Edition Yx 05)',
}


In [19]:
site = 'https://www.restoclub.ru'

In [6]:
def web_scraping(response, category, city):
    new_dataset = pd.DataFrame(
        columns=['name', 'category', 'address', 'hours', 'rating', 'avg_bill', 'middle_coffee_cup'])
    soup = BeautifulSoup(response.text, 'html.parser')
    cafes = soup.select('.page-search__list')
    for cafe in cafes[0].contents:
        if cafe.select_one('.search-place-title__name') is not None:
            name = cafe.select_one('.search-place-title__name').text
            print(name)
            if cafe.select_one('.search-place-card__info-item._bill') is not None:
                avg_bill_attrs = cafe.select_one('.search-place-card__info-item._bill').attrs
                bill = avg_bill_attrs['title']
            else:
                bill = ''

            link_attrs = cafe.select_one('.search-place-title__link').attrs
            link = site + link_attrs['href']
            category = category

            response_cafe = session.get(link)
            soup_cafe = BeautifulSoup(response_cafe.text, 'html.parser')

            if soup_cafe.select_one('.t-dotted') is not None:
                address = city + ', ' + soup_cafe.select_one('.t-dotted').text
            else:
                address = city + ', ' + soup_cafe.select_one('.text-decoration-underline').text
            if soup_cafe.select_one('.info__list.m5t') is not None:
                hours = ', '.join([time.text for time in soup_cafe.select_one('.info__list.m5t').contents])
            else:
                hours = ''

            if soup_cafe.select_one('.rating__value') is not None:
                rating = soup_cafe.select_one('.rating__value').text
            else:
                rating = ''

            new_record = pd.DataFrame([{'name': name, 'category': category, 'address': address, 'hours': hours,
                                        'rating': rating, 'avg_bill': bill}])
            new_dataset = pd.concat([new_dataset, new_record], ignore_index=True)

    return new_dataset

In [25]:
session = HTMLSession()
my_dataset = pd.DataFrame(columns=['name', 'category', 'address', 'hours', 'rating', 'avg_bill', 'middle_coffee_cup'])
response = requests.get('https://www.restoclub.ru/spb/search/kofejni-v-peterburge', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'кофейня', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/spb/search/italjanskie-restorany-v-peterburge', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'ресторан', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/spb/search/kitajskie-restorany-v-peterburge', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'ресторан', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/spb/search/restorany-russkoj-kuhni-v-peterburge', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'ресторан', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/spb/search/paby-sankt-peterburga', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/spb/search/bary-sankt-peterburga', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/spb/search/bary-sankt-peterburga', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Санкт-Петербург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/kofejnja-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'кофейня', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/piccerija-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'пиццерия', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/restorany-gruzinskoj-kuhni-v-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'ресторан', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/restorany-italjanskoj-kuhni-v-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'ресторан', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/japonskie-restorany-v-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'ресторан', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/bar-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/pivnye-restorany-bary-i-paby-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/vinoteki-s-somele-v-ekb', cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Екатеринбург')], ignore_index=True)

response = requests.get('https://www.restoclub.ru/ekb/search/restorany-kafe-i-bary-s-interesnymi-koktejaljami',
                        cookies=cookies,
                        headers=headers)
my_dataset = pd.concat([my_dataset, web_scraping(response, 'бар, паб', 'Екатеринбург')], ignore_index=True)

Theobroma
Астаманьяна
Caribia
Рок остров
Тройка
Midi Modi
Кофейня №1
Хороший день
Civil
Peche
Bonch
Нежно
Смена
Mer Kaffe
Вкус искусства
Gran
ТЧК
Mountain Coffee
Sibaristica
2A Concept
Coffee 3
Chloe Cafe
Щегол
Британские пекарни
Le Moniteur
Skuratov Coffee
Studio 212 Cafe
Чайникофф
1886 Bonomi
Coffee Room
Black coffee Co.
Pan&Cake
Pitcher
Etlon Coffee
Займемся кофе
Surf Coffee
Babe's Bakery
Bolshecoffee Roasters
Кофелософия
Характер кофе
Kiosk 59
Verle
Мой хлебный
Hogan Coffee
Coffee 22
Mr. Black
Fan of Donuts
Музей кофе
Знакомьтесь, Джо!
Кофеварим
Medici
Ripasso
Amo
Romeo's
Arancino
Tetto
Peperoni
Faces Cafe
Little Italy
Chelentano
Марчеллис
Mircuccio Osteria
Hype’s Pasta & Lounge
Ragu
CHE CAFE
Percorso
Sapori di Italia
Il Milanese
Mina
Bronte
Sapori di Italia
CHE CAFE
Sono Italiano
Сыроварня
Ponte
La Biga Italian bistro
Uva
Ognivo
Nomad
Bona Capona
Primi
Jerome
Toscana Grill
Mario Trattoria
Ponte
МА
Gianni
МА
Felichito
Bona Capona
Lorenzo Medici
Раскольников
Cafe People
Antico
Москв

In [26]:
my_dataset.shape

(698, 7)

In [27]:
my_dataset.to_csv('MyDataset.csv', index=False)

In [2]:
cookies_afisha = {
    'ruid': 'ugsAAH+dU2WWfeeAAVjGAAB=',
    '_ym_uid': '169997862617756492',
    '_ym_d': '1699978626',
    '_egguid': '01b27232-e5fc-cf65-9dd3-f2fd199a1d7b',
    '_eggDeviceUUID': '01b27232-e5fc-cf65-9dd3-f2fd199a1d7b',
    'city_id': '9',
    'HomeGeographyPlaceID': '9',
    '_ym_isad': '2',
    '_egg21_SessionUUID': '685dd0a7-38a0-4b57-ae3d-1818e5fdfc8b',
    '_ym_visorc': 'w',
}

headers_afisha = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'ruid=ugsAAH+dU2WWfeeAAVjGAAB=; _ym_uid=169997862617756492; _ym_d=1699978626; _egguid=01b27232-e5fc-cf65-9dd3-f2fd199a1d7b; _eggDeviceUUID=01b27232-e5fc-cf65-9dd3-f2fd199a1d7b; city_id=9; HomeGeographyPlaceID=9; _ym_isad=2; _egg21_SessionUUID=685dd0a7-38a0-4b57-ae3d-1818e5fdfc8b; _ym_visorc=w',
    'Pragma': 'no-cache',
    'Referer': 'https://www.afisha.ru/novosibirsk/restaurants/restaurant_list/page2/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 OPR/104.0.0.0 (Edition Yx 05)',
    'sec-ch-ua': '"Chromium";v="118", "Opera";v="104", "Not=A?Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

#response_afisha = requests.get('https://www.afisha.ru/novosibirsk/restaurants/restaurant_list/', cookies=cookies_afisha, headers=headers_afisha)

In [98]:
start_page = 1
end_page = 82

In [9]:
site_afisha = 'https://www.afisha.ru'

In [76]:
def clean_str(input):
    input = re.sub(r'\s*(\w+|\.)\s*', r' \1 ', input.replace('\n', ''))
    input = re.sub(r'\s+', ' ', input)
    return re.sub(r'\s*\.\s*', '.', input).strip()

In [83]:
def if_is_not_none(item):
    if item is not None:
        return item.text
    else:
        return ''

In [93]:
def web_scraping_afisha(site_afisha_get):
    count = 0
    new_dataset = pd.DataFrame(
        columns=['name', 'category', 'address', 'hours', 'rating', 'avg_bill', 'middle_coffee_cup'])

    for page_num in range(start_page, end_page + 1):
        params = {'page': page_num}
        response_afisha = requests.get(site_afisha_get, params=params, cookies=cookies_afisha,
                                       headers=headers_afisha)

        soup = BeautifulSoup(response_afisha.text, 'html.parser')
        cafes = soup.select('.list_wrap')

        for cafe in cafes[0].contents:
            if cafe.__eq__('\n'):
                continue
            link_attrs = cafe.select_one('.l-rest-card.places_cards').contents[1].attrs
            link = site_afisha + link_attrs['href']
            response_cafe = session_afisha.get(link)
            soup_cafe = BeautifulSoup(response_cafe.text, 'html.parser')

            name = clean_str(if_is_not_none(soup_cafe.select_one('.ficher-name')))
            count+=1
            print(f'{count}. {name}')

            category = clean_str(if_is_not_none(soup_cafe.select_one('.ficher-type')))
            if category.__eq__('Пивной ресторан'):
                category = 'бар, паб'
            address = clean_str(if_is_not_none(soup_cafe.select_one('.info-address__text')))
            rating = clean_str(if_is_not_none(soup_cafe.select_one('.rating_number')))
            hours = clean_str(if_is_not_none(soup_cafe.select_one('.contact_sense.js-contact-sense-opening-hours')))
            bill = clean_str(if_is_not_none(soup_cafe.select_one('.ficher-item').contents[7]))

            new_record = pd.DataFrame([{'name': name, 'category': category, 'address': address, 'hours': hours,
                                        'rating': rating, 'avg_bill': bill}])
            new_dataset = pd.concat([new_dataset, new_record], ignore_index=True)

    return new_dataset

In [88]:
session_afisha = HTMLSession()
my_dataset_afisha = pd.DataFrame(
    columns=['name', 'category', 'address', 'hours', 'rating', 'avg_bill', 'middle_coffee_cup'])
my_dataset_afisha = pd.concat([my_dataset_afisha,
                               web_scraping_afisha('https://www.afisha.ru/novosibirsk/restaurants/restaurant_list/')], ignore_index=True)

1. T.B.K.Lounge
2. Beerman & Grill
3. Clever
4. Коляда
5. Friends
6. Park Café
7. Ruby Aperitifs and Wines
8. Кардамон
9. Shisha
10. Capriccio
11. Pub 501
12. Puppen Haus
13. Баранжар
14. Ассоль
15. Баттерфляй
16. Кофемолка
17. Соседи
18. Ухо и медведь
19. Шемрок Restopub
20. Beerman & Пельмени
21. Сибирьсибирь
22. Агарта
23. Чашка кофе
24. Чашка кофе
25. Шинок у Вакулы
26. Якитория
27. St Patrick ' s Corner
28. Скоморохи
29. Старая Прага
30. Тифлис
31. 100 друзей
32. 10 Ten
33. 113
34. 22 Twenty Two
35. 2R
36. 33 пингвина
37. 33 пингвина
38. 33 пингвина
39. 33 пингвина
40. 33 пингвина
41. 33 пингвина
42. 33 пингвина
43. 33 пингвина
44. 33 пингвина
45. 33 пингвина
46. 33 пингвина
47. 33 пингвина
48. 33 пингвина
49. 33 пингвина
50. 55 широта
51. 5 звезд
52. 6 - 12 - 24
53. 68 чашек
54. 8 минут
55. Abnicum
56. Absolem
57. Alma
58. Amigo Churros
59. Amsterdam Pub
60. Arena Food
61. Art Coffee
62. Aurora
63. Avocado Queen
64. Aziatish
65. Banket Party
66. Barbara ждет
67. Barbaris
68. Bar 

In [89]:
my_dataset_afisha.shape

(1972, 7)

In [90]:
my_dataset_afisha.to_csv('MyDataset_afisha_novosib.csv', index=False)

In [94]:
my_dataset_afisha = pd.DataFrame(
    columns=['name', 'category', 'address', 'hours', 'rating', 'avg_bill', 'middle_coffee_cup'])
my_dataset_afisha = pd.concat([my_dataset_afisha,
                               web_scraping_afisha('https://www.afisha.ru/spb/restaurants/restaurant_list/')], ignore_index=True)

1. The Legends
2. Fresa ' s
3. Garage
4. Juno
5. Neon Bar
6. Prosa
7. Wine & Dine
8. Вах ! Вах !
9. Игристые
10. Duo
11. Бекицер
12. Русская рыбалка
13. Тепло
14. Zoom
15. Tartarbar
16. Duo Asia
17. Charlie
18. Italia
19. Траппист
20. O ' Hooligans
21. Пицца 22 см
22. Русская рыбалка
23. Пышечная
24. Чайный дом
25. Coffee 22
26. Centrale
27. Wood Bar
28. Co - op Garage
29. Camorra pizza e birra
30. The Hat Bar
31. Общество чистых тарелок
32. Schengen
33. El copitas
34. Venezia
35. Union Bar
36. Birch
37. Арка
38. Пельмения
39. Ботаника
40. R14
41. Graf - in
42. Smoke BBQ
43. Бричмула
44. Entrée
45. Red.Steak & Wine
46. Joly Woo
47. O ! Cuba
48. Кинг - понг
49. Hamlet + Jacks
50. Jérôme
51. Русская рюмочная № 1
52. Pita ' s
53. I ’ m Thankful for Today
54. Тандур
55. Capuletti
56. Блок
57. Чехов
58. Mickey & Monkeys
59. Dickens
60. Стирка 40 °
61. Dead Poets
62. Subzero
63. Big Wine Freaks
64. Buterbrodsky
65. Surf Coffee x G34
66. Terminal
67. Полторы комнаты
68. Сидрерия
69. 812 / Яру

In [95]:
my_dataset_afisha.shape

(1992, 7)

In [96]:
my_dataset_afisha.to_csv('MyDataset_afisha_spb.csv', index=False)